In [11]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from tqdm import tqdm_notebook as tqdm
import math
import numpy as np
import pandas as pd
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt

from scipy.cluster.hierarchy import linkage, dendrogram

In [ ]:
import glob
from bs4 import BeautifulSoup

files = glob.glob('./work/*html')

def parse(fileName):
    with open(fileName) as f:
        soup = BeautifulSoup(f, 'html.parser')

    title      = soup.select_one('article.blog-entry-article h1.blog-title').get_text()
    date       = soup.select_one('article.blog-entry-article div.blog-date').get_text()
    category   = soup.select_one('article.blog-entry-article li.blog-category').get_text()
    text       = soup.select_one('article.blog-entry-article div.content').get_text()
    
    
    return [fileName, title, date, category, text]


data = [parse(fileName) for fileName in tqdm(files)]

In [ ]:
df = pd.DataFrame(data, columns=['file', 'title', 'date', 'category', 'text'])
df.head()

In [56]:
print(m.parse(df['text'].loc[1]))

*,扱い,アツカイ,アツカイ
を	助詞,格助詞,一般,*,*,*,を,ヲ,ヲ
分ける	動詞,自立,*,*,一段,基本形,分ける,ワケル,ワケル
ため	名詞,非自立,副詞可能,*,*,*,ため,タメ,タメ
、	記号,読点,*,*,*,*,、,、,、
コンテナ	名詞,一般,*,*,*,*,コンテナ,コンテナ,コンテナ
ログ	名詞,サ変接続,*,*,*,*,ログ,ログ,ログ
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
kubernetes	名詞,固有名詞,組織,*,*,*,*
.*	名詞,サ変接続,*,*,*,*,*
で	助詞,格助詞,一般,*,*,*,で,デ,デ
タグ	名詞,一般,*,*,*,*,タグ,タグ,タグ
付け	名詞,一般,*,*,*,*,付け,ツケ,ツケ
する	動詞,自立,*,*,サ変・スル,基本形,する,スル,スル
。	記号,句点,*,*,*,*,。,。,。
enable	名詞,一般,*,*,*,*,*
_	名詞,サ変接続,*,*,*,*,*
ruby	名詞,一般,*,*,*,*,*
true	名詞,固有名詞,組織,*,*,*,*
により	助詞,格助詞,連語,*,*,*,により,ニヨリ,ニヨリ
、	記号,読点,*,*,*,*,、,、,、
${...}	名詞,サ変接続,*,*,*,*,*
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
ruby	名詞,固有名詞,組織,*,*,*,*
の	助詞,連体化,*,*,*,*,の,ノ,ノ
式	名詞,一般,*,*,*,*,式,シキ,シキ
として	助詞,格助詞,連語,*,*,*,として,トシテ,トシテ
評価	名詞,サ変接続,*,*,*,*,評価,ヒョウカ,ヒョーカ
さ	動詞,自立,*,*,サ変・スル,未然レル接続,する,サ,サ
れる	動詞,接尾,*,*,一段,基本形,れる,レル,レル
。	記号,句点,*,*,*,*,。,。,。
log	名詞,一般,*,*,*,*,*
_	名詞,サ変接続,*,*,*,*,*
group	名詞,一般,*,*,*,*,*
は	助詞,係助詞,*,*,*,*,は,ハ,ワ
以下	名詞,非自立,副詞可能,*,*,*,以下,イカ,イカ
の	助詞,連体化,*,*,*,*,の,ノ,ノ
5	名詞,数,*,*,*,*,*
要素	名詞,一般,*,*,*,*,要素,ヨウソ,ヨーソ
を	助詞,格助詞,一般,*,*,*,を

In [57]:
## 単語の抜き出し

import MeCab
m = MeCab.Tagger('-d /usr/local/lib/mecab/dic/ipadic')

def parseText(text):
    node = m.parseToNode(text)
    words = []
    while node:
        fields = node.feature.split(",")
        # if fields[0] in ['名詞', '動詞', '形容詞']:
        if fields[0] in ['名詞'] and len(node.surface) > 1:
            words.append(node.surface)
        node = node.next
    
    return words

In [58]:
df['words'] = df['text'].map(lambda text: parseText(text))

In [59]:
df['words'].head()

0    [ソフトウェア, エンジニア, hota, 今回, ソフトウェア, エンジニア, 採用, お...
1    [ソフトウェア, エンジニア, skirino, 最近, コンテナ, アプリケーション, 設...
2    [ソフトウェア, エンジニア, 田中, 系列, データ, event, time, proc...
3    [プロダクト, マネージャー, 横井, 啓介, 前回, 投稿, デジタルトランスフォーメーシ...
4    [FLYWHEEL, ソフトウェア, エンジニア, saoi, 前回, 投稿, 投稿, FL...
Name: words, dtype: object

In [60]:
import gensim
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

## 辞書とコーパスの作成

NUM_TOPICS = 5

dictionary = Dictionary(df['words'])
dictionary.filter_extremes(no_below=3, no_above=0.8)
corpus = [dictionary.doc2bow(words) for words in df['words']]

In [61]:
start = 2
limit = 22
step = 1

coherence_vals = []
perplexity_vals = []

for n_topic in tqdm(range(start, limit, step)):
    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=n_topic, random_state=0)
    perplexity_vals.append(np.exp2(-lda_model.log_perplexity(corpus)))
    coherence_model_lda = gensim.models.CoherenceModel(model=lda_model, texts=df['words'], dictionary=dictionary, coherence='c_v')
    coherence_vals.append(coherence_model_lda.get_coherence())

  0%|          | 0/20 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# evaluation
x = range(start, limit, step)

fig, ax1 = plt.subplots(figsize=(12,5))

# coherence
c1 = 'darkturquoise'
ax1.plot(x, coherence_vals, 'o-', color=c1)
ax1.set_xlabel('Num Topics')
ax1.set_ylabel('Coherence', color=c1); ax1.tick_params('y', colors=c1)

# perplexity
c2 = 'slategray'
ax2 = ax1.twinx()
ax2.plot(x, perplexity_vals, 'o-', color=c2)
ax2.set_ylabel('Perplexity', color=c2); ax2.tick_params('y', colors=c2)

# Vis
ax1.set_xticks(x)
fig.tight_layout()
plt.show()
plt.savefig('metrics.png')


In [62]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, random_state=0)
lda_model.save('lda.model')

In [63]:
### 描画

import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()

from wordcloud import WordCloud


In [67]:
# WordCloud
fig, axs = plt.subplots(ncols=2, nrows=math.ceil(lda_model.num_topics/2), figsize=(16,20))
axs = axs.flatten()

def color_func(word, font_size, position, orientation, random_state, font_path):
    return 'darkturquoise'

for i, t in enumerate(range(lda_model.num_topics)):

    x = dict(lda_model.show_topic(t, 30))
    im = WordCloud(
        background_color='black',
        color_func=color_func,
        max_words=4000,
        width=300, height=300,
        random_state=0
    ).generate_from_frequencies(x)
    axs[i].imshow(im.recolor(colormap= 'Paired_r' , random_state=244), alpha=0.98)
    axs[i].axis('off')
    axs[i].set_title('Topic '+str(t))

# vis
plt.tight_layout()
plt.show()

# save as png
plt.savefig('wordcloud.png') 

In [66]:
matplotlib.matplotlib_fname()

'/Users/tune/.pyenv/versions/flywheel/lib/python3.8/site-packages/matplotlib/mpl-data/matplotlibrc'

In [14]:
plt.rcParams["font.family"] = 'IPAexGothic'
print(plt.rcParams["font.family"]) # => ['sans-serif']
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(range(10),range(10), label="データ1")
ax.plot(range(10),range(0,20,2), label="データ2")
ax.set_title("タイトル")
ax.set_xlabel("x軸のラベル")
ax.set_ylabel("y軸のラベル")
ax.legend()
plt.show()
plt.savefig('out1.png')

['IPAexGothic']


In [ ]:
from collections import defaultdict
from gensim.models.keyedvectors import KeyedVectors
from sklearn.cluster import KMeans

In [ ]:
# 以下から最新の学習済みモデルをダウンロード
# https://github.com/singletongue/WikiEntVec/releases
# 今回利用したのは20190520のjawiki.all_vectors.100d.txt.bz2

model = KeyedVectors.load_word2vec_format('work/jawiki.all_vectors.100d.txt')

In [ ]:
word = '理科'
results = model.wv.most_similar(word)
print(word, "と類似度の高い単語") 
for result in results:
    print(result)